# Pipeline et modèle

In [1]:
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.impute import KNNImputer

import pickle

from sklearn.compose import make_column_selector, ColumnTransformer, make_column_transformer
# sklearn.compose: The sklearn.compose module is a submodule of the sklearn library for machine learning in Python. It provides functions for creating complex preprocessing and modeling pipelines.
from sklearn.preprocessing import OneHotEncoder,StandardScaler,PolynomialFeatures,RobustScaler
#sklearn.preprocessing: The sklearn.preprocessing module is a submodule of the sklearn library that provides functions for preprocessing data, such as scaling and normalizing features, imputing missing values, and encoding categorical variables.
from sklearn.linear_model import Ridge,LinearRegression,Lasso, ElasticNet
# sklearn.linear_model: The sklearn.linear_model module is a submodule of the sklearn library that provides functions for fitting linear models for regression and classification.
from sklearn.pipeline import make_pipeline
# sklearn.pipeline: The sklearn.pipeline module is a submodule of the sklearn library that provides functions for creating and working with pipelines of transformers and models.
from sklearn.model_selection import train_test_split,GridSearchCV,learning_curve, RandomizedSearchCV, cross_val_score, KFold
# sklearn.model_selection: The sklearn.model_selection module is a submodule of the sklearn library that provides functions for splitting data into training and test sets, evaluating models using cross-validation, and hyperparameter tuning.
from sklearn.dummy import DummyRegressor
# sklearn.dummy: The sklearn.dummy module is a submodule of the sklearn library that provides simple dummy models for regression and classification.

from sklearn.metrics import *
from scipy.stats import probplot
import matplotlib.pyplot as plt
import seaborn as sns
from myfunctions import *
from statistics import mean

In [2]:
df = pd.read_csv("../data/new_AmesHousing.csv")

In [3]:
df.shape

(2921, 11)

In [4]:
#Imputation
df = KNNImputer_imputation(df,['Bsmt Qual', 'Garage Finish'])

In [5]:
numeric_features = ["TotalSF", "Overall Qual", "Garage Cars", "TotalBathrooms", "Age"]
ordinal_features = [ "Bsmt Qual",  "Exter Qual", "Kitchen Qual"]
#cat_feature = ["Neighborhood", "Foundation"]
cat_feature = ["Neighborhood", "Garage Finish"]
all_col = numeric_features.copy()
all_col.extend(ordinal_features)
all_col.extend(cat_feature)

X = df[all_col]
y = df[["SalePrice"]]


In [6]:
def make_pipeline_to_ML(X, y, ordinal_features=None):
    X_train, X_test, y_train, y_test = train_test_split(
                                                        X, y, shuffle=True, 
                                                        train_size=0.8, 
                                                        random_state=42, 
                                                        stratify=X[['Overall Qual']]
                                                        )

    numerical_features = make_column_selector(dtype_include=np.number)
    numerical_pipeline = make_pipeline(StandardScaler(with_mean=False))


    categorical_features = make_column_selector(dtype_exclude=np.number)
    categorical_pipeline = make_pipeline(OneHotEncoder(handle_unknown='ignore'))

    ordinal_pipeline = make_pipeline(OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_pipeline, numerical_features),
            ('cat', categorical_pipeline, categorical_features),
            ('ord', ordinal_pipeline, ordinal_features) if ordinal_pipeline else None
        ]
    )

    return preprocessor, X_train, X_test, y_train, y_test

In [7]:

preprocessor, X_train, X_test, y_train, y_test = make_pipeline_to_ML(X, y, ordinal_features=ordinal_features)


In [8]:
################################
################################
#
# Get best Hypermarameters
#
################################
################################


# get_best_params(
#    PolynomialFeatures_degree = 1,  
#    model=Lasso(max_iter=100000, 
#    tol=0.001,
#    random_state=42, 
#    selection='cyclic'),   
#    param_grid= {'lasso__alpha': np.linspace(0.01,50,20)}, 
#    preprocessor = preprocessor, 
#    X_train = X_train, 
#    y_train = y_train)


# param_grid= {'lasso__alpha': np.linspace(0.1,1,20)}
# get_best_params_Lasso(param_grid= param_grid, 
#                       preprocessor = preprocessor, 
#                       X_train = X_train, 
#                       y_train = y_train , 
#                       y_test = y_test, 
#                       X_test = X_test)

In [9]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
import numpy as np

reg = Lasso()
pipe = Pipeline([
     ('preprocessor', preprocessor),
     ('reg', reg)
])
param_grid = {'reg__alpha': np.linspace(0.0001, 0.0002, 200)}  # Use 'reg__alpha' instead of 'alpha'
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, cv=skf.split(X_train, X_train['Kitchen Qual']), n_jobs=-1, verbose=0)
# Fit the GridSearchCV to find the best parameters
grid_search.fit(X_train, y_train)

# Get the best parameters and best model from the GridSearchCV
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Use the best model to make predictions on the test data (X_test)
y_pred = best_model.predict(X_test)

# Calculate the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

print("Best Parameters:", best_params)
print("MAE:", mae)


/home/apprenant/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 214982539718.34454, tolerance: 1183488793.3511915
  model = cd_fast.sparse_enet_coordinate_descent(
/home/apprenant/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 200920817257.28082, tolerance: 1110539568.0925162
  model = cd_fast.sparse_enet_coordinate_descent(
/home/apprenant/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 228607368502.16034, tolerance: 1175837992.335369
  model = cd_fast.sparse_enet_coordinate_descent(
/home/apprenant/.local/lib/python3.10/site-packages/skle

Best Parameters: {'reg__alpha': 0.0002}
MAE: 17100.47861597285


/home/apprenant/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 269262992133.42163, tolerance: 1461944407.6689343
  model = cd_fast.sparse_enet_coordinate_descent(


In [10]:
R2, MAE, RMSE, Model_score_test, Model_score_training, scores_mean, scores_std, LR_model_1 = Lasso_with_CV(PolynomialFeatures_degree = 1, 
                    best_alpha = 1, X_train = X_train, y_train = y_train, X_test = X_test, y_test = y_test, 
                    preprocessor = preprocessor, shuffle=True, random_state=42,
                    isplot = False, isinfo = True, include_learning_curve = False) 

/home/apprenant/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 72719167672.03064, tolerance: 1095482192.4329813
  model = cd_fast.sparse_enet_coordinate_descent(
/home/apprenant/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12693655718.340454, tolerance: 1181114489.4722946
  model = cd_fast.sparse_enet_coordinate_descent(
/home/apprenant/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 81051310004.79187, tolerance: 1220293061.7438784
  model = cd_fast.sparse_enet_coordinate_descent(
/home/apprenant/.local/lib/python3.10/site-packages/sklea

LR with Kfold CV (Polynomial degree=1)

[0.90726718 0.887562   0.88349222 0.89870857 0.8661695 ]
0.8886398909245644
CV Accuracy: 0.89 (+/- 0.01)
R2: 0.8911
MAE: 17092.7154
RMSE: 25643.29
Model_score_test: 25643.29
Model_score_training: 25643.29


/home/apprenant/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10941379908.983887, tolerance: 1461944407.6689343
  model = cd_fast.sparse_enet_coordinate_descent(


In [11]:
reg = Lasso(alpha=1)
pipe = Pipeline([
     ('preprocessor', preprocessor),
     ('reg', reg)
])

pipe.fit(X_train, y_train) 

/home/apprenant/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10941379908.983887, tolerance: 1461944407.6689343
  model = cd_fast.sparse_enet_coordinate_descent(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler(with_mean=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fa644767bb0>),
                                                 ('cat',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fa5ef83baf0>),
                                                 ('ord',
                                                  Pipeline(steps=[('ordinalencoder',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1))]),
                                                  ['Bsmt Qual', 'Exter Qual',
                                                   'Kitchen Qual'])])),
                ('reg', Lasso(alpha=1))])

In [12]:
pipe.score(X_test,y_test)

predict_train  = pipe.predict(X_train)
predict_test  = pipe.predict(X_test)

# Root Mean Squared Error on train and test date
print('MAE on train data: ', mean_absolute_error(y_train, predict_train))
print('MAE on test data: ',  mean_absolute_error(y_test, predict_test))

MAE on train data:  17835.959517372237
MAE on test data:  17092.71542644749


In [13]:
filename = '../real_estate_app/main_app/static/models/finalized_model.pkl'
pickle.dump(pipe, open(filename, 'wb'))

In [14]:
df.columns

Index(['TotalSF', 'Overall Qual', 'Neighborhood', 'Bsmt Qual', 'Exter Qual',
       'Kitchen Qual', 'Garage Cars', 'TotalBathrooms', 'Age', 'Garage Finish',
       'SalePrice'],
      dtype='object')

In [17]:
import pandas as pd

dico = {
    'TotalSF': 80,
    'Overall Qual': 7,
    'Garage Cars': 2,
    'TotalBathrooms': 2,
    'Age': 23,
    'Neighborhood': 'Gilbert',
    'Bsmt Qual': 'Gd',
    'Exter Qual': 'Po',
    'Kitchen Qual': 'Gd',
    'Garage Finish': 'Fin',
}

# Convert the dictionary to a pandas DataFrame
input_df = pd.DataFrame([dico])

# Use the trained pipeline to make predictions
predicted_values = pipe.predict(input_df)


In [21]:
predicted_values[0]

136543.37377664383